In [1]:
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import multivariate_normal
from sklearn.preprocessing import normalize
from scipy.misc import logsumexp
from scipy.linalg import inv
from scipy.linalg import pinv

def fit(Y,K,p,labels,max_iter):
    #p is the degree of the polynomial regression
    #K is the number of classes
    #Y is a n*d obeservation matrix
    #labels is the labels matrix of Y
    
    """ Initialisation """
    n,d = Y.shape
    pf = PolynomialFeatures(degree=p) #transformer to get polynomial features of the time
    t = pf.fit_transform(np.arange(n).reshape(-1,1)) #matrix containing the polynomial features of the time
    pi = (1.0/K)*np.ones((K))
    A = (1.0/(2.0*(K-1)))*np.ones((K,K))
    for i in range(K):
        A[i,i] = 0.5
    B = [] #list of coefficient matrices
    sigma = [] #list of covariance matrices 
    for i in range(K):
        B.append(np.random.uniform(-1,1,(p+1,d)))
        s = np.transpose(Y[labels==i,:])
        sigma.append(np.cov(s))
        #sigma.append(np.eye(d))
    F = np.zeros((n,K))
    for i in range(n):
        for j in range(K):
            mean = np.dot(B[j].T,t[j,:])
            F[i,j] = multivariate_normal(mean=mean,cov=sigma[j],allow_singular=True).pdf(Y[i,:])
    for i in range(max_iter):
        """ E step """
        #appeler tranpose(F) :c
        #calculate P1 and P2
        #P1 is a n*K matrix
        #P2 is a matrix of size n*K*K
        P11,P2 = fwd_bkw(pi,np.transpose(F),A)
        P1 = np.transpose(P11)
        """ M step """
        pi = P1[0,:]
        A = np.transpose(np.sum(P2[1:,:,:],axis=0))
        for i in range(K):
            A[i,:] = A[i,:]/np.sum(P1[1:,:],axis=0)
            W = np.diag(P1[:,i])
            M = np.dot(np.transpose(t),np.dot(W,t))
            m = M.shape[0]
            B[i] = np.dot(pinv(M),np.transpose(t))
            B[i] = np.dot(B[i],np.dot(W,Y))
            c = 1.0/np.sum(P1[:,i])
            M = Y-np.dot(t,B[i])
            sigma[i] = c*np.dot(np.transpose(M),np.dot(W,M))
        #calculate the Bs and the sigmas
        """ Update F """
        for i in range(n):
            for j in range(K):
                mean = np.dot(B[j].T,t[j,:])
                F[i,j] = multivariate_normal(mean=mean,cov=sigma[j],allow_singular=True).pdf(Y[i,:])
    return pi,A,B,sigma

def fwd_bkw(init_dist,F,A):
    [K,T] = F.shape
    alpha = np.zeros((K,T))
    beta = np.zeros((K,T))
    
    # Forward recursion
    t=1
    alpha[:,t] = np.multiply(init_dist,F[:,t])
    alpha[:,t] = normalize(alpha[:,t].reshape(1,-1),norm='l1')
    
    for t in np.arange(2,T):
        m = np.dot(A.T,alpha[:,t-1])
        alpha[:,t]=np.multiply(m,F[:,t])
        alpha[:,t] = normalize(alpha[:,t].reshape(1,-1),norm='l1')
    
    # Backward recursion
    beta[:,T-1] = normalize(np.ones(K).reshape(1,-1),norm='l1')
    for t in np.arange(1,T-1)[::-1]:
        b = np.multiply(beta[:,t+1], F[:,t+1])
        beta[:,t] = np.dot(A,b)
        beta[:,t] = normalize(beta[:,t].reshape(1,-1),norm='l1')
    
    # Compute P1=p(kt|y1,...,yT)=alpha*beta/sum(alpha*beta)
    P1 = np.zeros((K,T))
    for t in np.arange(T):
        P1[:,t] = np.multiply(alpha[:,t],beta[:,t])/logsumexp(np.multiply(alpha[:,t],beta[:,t]))
        P1[:,t] = normalize(P1[:,t].reshape(1,-1),norm='l1')
    
    # Compute P2=p(kt,kt+1|y1,...,yT)
    P2 = np.zeros((T-1,K,K))
    for t in np.arange(1,T-2):
        for i in np.arange(1,K-1):
            for j in np.arange(1,K-1):
                P2[t,i,j]=alpha[i,t]*beta[j,t+1]*F[j,t+1]
        P2[t,:,:] = normalize(P2[t,:,:],norm='l1')
    return P1,P2

In [3]:
import numpy as np
A = np.zeros((2,3))

In [23]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree=3)

In [28]:
pf.fit_transform(np.arange(3).reshape(-1,1))

array([[ 1.,  0.,  0.,  0.],
       [ 1.,  1.,  1.,  1.],
       [ 1.,  2.,  4.,  8.]])

In [37]:
from scipy.stats import multivariate_normal

0.3989422804014327

In [61]:
np.linalg.inv(np.diag(np.ones(3)))

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [10]:
fit(np.random.randn((100,25)),3,5,np.random.randint(0,4,size=100),100)

TypeError: an integer is required

In [18]:
np.arange(2,6)

array([2, 3, 4, 5])